In [1]:
import pandas as pd

In [2]:
candidatos = pd.read_csv('Resultados\\Candidatos.csv', delimiter=',', encoding='utf-8')
candidatos.head()

,ANO_ELEICAO,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NR_PARTIDO,SG_PARTIDO,PASSOU_TURNO
0,2022,BA,GOVERNADOR,50001609248,ANTONIO CARLOS PEIXOTO DE MAGALHÃES NETO,44,UNIÃO,True
1,2022,PB,GOVERNADOR,150001621388,PEDRO OLIVEIRA CUNHA LIMA,45,PSDB,True
2,2022,SC,GOVERNADOR,240001611127,JORGINHO DOS SANTOS MELLO,22,PL,True
3,2022,MS,GOVERNADOR,120001714229,RENAN BARBOSA CONTAR,28,PRTB,True
4,2022,SC,GOVERNADOR,240001647445,DÉCIO NERY DE LIMA,13,PT,True


In [3]:
despesas = pd.read_csv('Resultados\\Despesas.csv', delimiter=',', encoding='utf-8')
despesas.head()

,NR_TURNO,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,SG_PARTIDO,DS_ORIGEM_DESPESA,VR_DESPESA_CONTRATADA
0,1,AC,GOVERNADOR,10001639519,CYLMARA FERNANDES DA ROCHA GRIPP,MDB,Atividades de militância e mobilização de rua,"1858,40"
1,1,AC,GOVERNADOR,10001612476,GLADSON DE LIMA CAMELI,PP,Atividades de militância e mobilização de rua,"1300,00"
2,1,AC,GOVERNADOR,10001639519,CYLMARA FERNANDES DA ROCHA GRIPP,MDB,Atividades de militância e mobilização de rua,"848,00"
3,1,AC,GOVERNADOR,10001623440,MARCIO MIGUEL BITTAR,UNIÃO,Atividades de militância e mobilização de rua,"1200,00"
4,1,AC,GOVERNADOR,10001639519,CYLMARA FERNANDES DA ROCHA GRIPP,MDB,Atividades de militância e mobilização de rua,"1857,00"


In [4]:
votos = pd.read_csv('Resultados\\Votos.csv', delimiter=',', encoding='utf-8')
votos.head()

,NR_TURNO,SG_UF,DS_CARGO,QT_VOTOS,SQ_CANDIDATO
0,1,AC,GOVERNADOR,1,10001595336
1,1,AC,GOVERNADOR,2,10001595336
2,1,AC,GOVERNADOR,1,10001595336
3,1,AC,GOVERNADOR,2,10001595336
4,1,AC,GOVERNADOR,1,10001595336


In [ ]:
dimensao_candidato = candidatos.drop(columns=["ANO_ELEICAO", "SG_UF", "DS_CARGO", "NR_PARTIDO", "PASSOU_TURNO"])

dimensao_candidato.to_csv('Estrela\\Dimensao_Candidatos.csv', index=False)

In [25]:
dimensao_local = candidatos.drop(columns=["ANO_ELEICAO", "NM_CANDIDATO", "DS_CARGO", "NR_PARTIDO", "PASSOU_TURNO", "SG_PARTIDO", "SQ_CANDIDATO"])

dimensao_local = dimensao_local.drop_duplicates(subset=["SG_UF"])
dimensao_local = dimensao_local.reset_index(drop=False).rename(columns={'index': 'COD_UF'}).sort_values(by='SG_UF', ascending=True)

dimensao_local.to_csv('Estrela\\Dimensao_Local.csv', index=False)

In [44]:
dimensao_despesa = despesas.drop(columns=["NR_TURNO", "SG_UF", "SQ_CANDIDATO", "DS_CARGO", "NM_CANDIDATO", "SG_PARTIDO", "VR_DESPESA_CONTRATADA"])

dimensao_despesa = dimensao_despesa.drop_duplicates()
dimensao_despesa = dimensao_despesa.reset_index(drop=False).rename(columns={'index': 'COD_DP'})

dimensao_despesa.to_csv('Estrela\\Dimensao_Despesas.csv', index=False)


In [42]:
dimensao_turno = votos.drop(columns=["SG_UF", "DS_CARGO", "QT_VOTOS", "SQ_CANDIDATO"])

dimensao_turno = dimensao_turno.drop_duplicates()
dimensao_turno.loc[len(dimensao_turno)] = "Ambos"
dimensao_turno = dimensao_turno.reset_index(drop=False).rename(columns={'index': 'COD_TURNO'})

dimensao_turno.to_csv("Estrela\\Dimensao_Turno.csv", index=False)

In [51]:
fatos_votos = votos.merge(dimensao_local[['SG_UF', 'COD_UF']], how='left', on='SG_UF')
fatos_votos = fatos_votos.merge(dimensao_turno[['NR_TURNO', 'COD_TURNO']], how='left', on='NR_TURNO')

fatos_votos = fatos_votos.drop(columns=['SG_UF', 'NR_TURNO'])

fatos_votos = fatos_votos[['COD_TURNO', 'COD_UF', 'QT_VOTOS', 'SQ_CANDIDATO']]

fatos_votos.to_csv('Estrela\\Fatos_Votos.csv', index=False)

In [54]:
fatos_despsas = despesas.merge(dimensao_local[['SG_UF', 'COD_UF']], how='left', on='SG_UF')

fatos_despsas = fatos_despsas.merge(dimensao_turno[['NR_TURNO', 'COD_TURNO']], how='left', on='NR_TURNO')
fatos_despsas = fatos_despsas.merge(dimensao_despesa[['DS_ORIGEM_DESPESA', 'COD_DP']], how='left', on='DS_ORIGEM_DESPESA')

fatos_despsas = fatos_despsas.drop(columns=['NR_TURNO', 'SG_UF', 'DS_ORIGEM_DESPESA', 'NM_CANDIDATO', 'SG_PARTIDO'])
fatos_despsas = fatos_despsas[['COD_TURNO', 'COD_UF', 'COD_DP', 'SQ_CANDIDATO', 'VR_DESPESA_CONTRATADA']]

fatos_despsas['VR_DESPESA_CONTRATADA'] = fatos_despsas['VR_DESPESA_CONTRATADA'].replace({'\D': '', ',': '.'}, regex=True).astype(int)

fatos_despsas.to_csv('Estrela\\Fatos_Despesas.csv', index=False)

<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
C:\Users\andre\AppData\Local\Temp\ipykernel_17792\2287610646.py:9: SyntaxWarning: invalid escape sequence '\D'
  fatos_despsas['VR_DESPESA_CONTRATADA'] = fatos_despsas['VR_DESPESA_CONTRATADA'].replace({'\D': '', ',': '.'}, regex=True).astype(int)
